In [50]:
import time
import random
from collections import deque

In [51]:
def load_graph_from_mtx(filename):
    graph = {}
    with open(filename, 'r') as file:
        for i, line in enumerate(file):
            # preskocimo prva tri reda (jer su tu podaci o broju grana, cvorova...)
            if i < 3:
                continue
            
            nodes = line.strip().split()
            node1, node2 = int(nodes[0]), int(nodes[1])

            if node1 not in graph:
                graph[node1] = set()
            if node2 not in graph:
                graph[node2] = set()

            # Dodavanje suseda
            graph[node1].add(node2)
            graph[node2].add(node1)

    return graph


In [52]:
def bfs_shortest_paths(graf, start):
    dist = {node: float('inf') for node in graf} 
    dist[start] = 0
    queue = deque([start])
    paths = {node: [] for node in graf}
    paths[start] = [start]
    
    while queue:
        current = queue.popleft()
        
        for neighbor in graf[current]:
            if dist[neighbor] == float('inf'):
                dist[neighbor] = dist[current] + 1
                queue.append(neighbor)
            
            if dist[neighbor] == dist[current] + 1:
                paths[neighbor].append(current)
    
    return dist, paths

def calculate_betweenness(graf):
    betweenness = {node: 0 for node in graf}
    
    for start in graf:
        dist, paths = bfs_shortest_paths(graf, start)
        
        for node in graf:
            if node == start:
                continue
            
            for neighbor in paths[node]:
                if neighbor != start:
                    betweenness[neighbor] += 1
    
    return betweenness

def betweenness_centralnost(cvor, betweenness):
    return betweenness.get(cvor, 0) 

In [53]:
def greedy_burning(graf):
    
    S=[] # sekvenca zapaljenih cvoreva
    B= set() # skup zapaljenih cvoreva
    NB= set(graf.keys()) # skup nezapaljenih cvoreva, inicajlno su to svi cvorevi grafa
    
    # ako imamo vise kandidata, biramo onog sa najvisim betweenness
    najveci_stepen = max(len(graf[node]) for node in graf)
    kandidati = [node for node in graf if len(graf[node]) == najveci_stepen]
    trenutni_cvor = None
    
    if len(kandidati) > 1:
        betweenness = calculate_betweenness(graf)
        trenutni_cvor = max(kandidati, key=lambda node: betweenness_centralnost(node, betweenness))
    else:
        trenutni_cvor = kandidati[0]
        
    print("pocetni:",trenutni_cvor)
    S.append(trenutni_cvor)
    B.add(trenutni_cvor)
    NB.remove(trenutni_cvor)
    
    
    while len(B)<len(graf):
       
        # korak 2: zapaliti sve susjede (koji nisu zapaljeni) od cvorova u skupu zapaljenih, dodajemo ih u B, brisemo iz NB
        novozapaljeni = []
        for cvor in B:
            susjedi = graf[cvor]  # svi susjedi trenutnog zapaljenog cvora
            for susjed in susjedi:
                if susjed in NB:  # ako susjed nije zapaljen
                    novozapaljeni.append(susjed)  # dodajemo ga u zapaljene
                    NB.remove(susjed)  
        
        for nz in novozapaljeni:
            B.add(nz) #dodajemo sve susjede u zapaljene
        
        print("nakon prvog koraka u petlji")
        print("sekvenca>",S)
        print("zapaljeni:",B)
        print("nezapaljeni",NB)
        
        # pravimo listu onih cvorova za koje znamo da ce se u iducem koraku zapaliti. Bolje da izaberemo sljedeci direktni
        # cvor za koji znamo da nece biti zapaljen u narednom koraku
        buducizapaljeni = []
        for b in B:
            susjedi_b = graf[b]
            for susjed in susjedi_b:
                if susjed in NB:
                    if susjed not in buducizapaljeni:
                        buducizapaljeni.append(susjed)
        print('-----')
        print("buduci:",buducizapaljeni)
        print("br buduci:",len(buducizapaljeni))
        print('-----')
        
        
        # korak 3: pronaci iduci cvor koji ce biti direktno zapaljen, dodati ga u S, B i izbrisati iz NB
        
        max_broj_zapaljenih = -1
        najbolji_cvor = None
        kandidati_za_najboljeg = []
        
        if NB:
            for cvor in NB:
                if cvor not in buducizapaljeni: 
                    nezapaljeni_susjedi = len([n for n in graf[cvor] if n not in B and n not in buducizapaljeni])
                    if nezapaljeni_susjedi>max_broj_zapaljenih:
                        max_broj_zapaljenih = nezapaljeni_susjedi
                        
            for cvor in NB:
                if cvor not in buducizapaljeni:
                    nezapaljeni_susjedi = len([n for n in graf[cvor] if n not in B and n not in buducizapaljeni])
                    if nezapaljeni_susjedi == max_broj_zapaljenih:
                        kandidati_za_najboljeg.append(cvor) # u slucaju da imamo vise kandidata za najbolji cvor
                        
            print("kandidati:", kandidati_za_najboljeg)
            
            if len(kandidati_za_najboljeg)>1:# izbor na osnovu betweenness centralnosti
                 najbolji_cvor = max(kandidati_za_najboljeg, key=lambda cvor: betweenness_centralnost(cvor, calculate_betweenness(graf)))
            elif len(kandidati_za_najboljeg)==1:
                najbolji_cvor = kandidati_za_najboljeg[0]
                
            if najbolji_cvor is not None:
                S.append(najbolji_cvor)
                B.add(najbolji_cvor)
                NB.remove(najbolji_cvor)
            else:
             # Ako nema nijednog najboljeg cvora znaci da su svi preostali cvorovi povezani i mogu biti zapaljeni
             # Dodajemo slucajni cvor iz preostalih nezapaljenih čvorova
                random_cvor = random.choice(list(NB))
                S.append(random_cvor)
                B.add(random_cvor)
                NB.remove(random_cvor)
            print("naj", najbolji_cvor)
        else:
            break
        
        
        
        print("------")
        print("sekvenca>",S)
        print("zapaljeni:",B)
        print("nezapaljeni",NB)
        print("br zapaljenih:",len(B))
        print("br nezapaljenih:",len(NB))
        
        
    return S
        

In [54]:
import random

def is_burning_sequence(graf, S):
    """
    Proverava da li je data sekvenca S validna burning sekvenca prema pravilima.
    """
    zapaljeni = set()
    for i<=len(S)+1:
        for cvor in S:
        if i == 0:
            # Korak 1: Pali se prvi cvor
            zapaljeni.add(cvor)
        else:
            # Korak 2 i 3: Pale se svi susjedi zapaljenih čvorova i pali se sledeći cvor direktno
            for zapaljeni_cvor in list(zapaljeni):
                for susjed in graf[zapaljeni_cvor]:
                    if susjed not in list(zapaljeni):
                        zapaljeni.add(susjed)  # Pale se susedi zapaljenih čvorova
            zapaljeni.add(cvor)  # Pali se sledeći cvor direktno
        
        # Ako sekvenca pokrije ceo graf, odmah možemo završiti
        if len(zapaljeni) == len(graf):
            return True
    
    return False


def local_search(graf, S):
    """
    Lokalna pretraga koja pokušava da poboljša sekvencu S.
    """
    najbolja_sekvenca = S
    # Proveravamo sekvencu S
    if not is_burning_sequence(graf, S):
        return najbolja_sekvenca  # Ako sekvenca nije validna burning sekvenca, ne menja ništa

    najbolji_rezultat = evaluate_solution(graf, S)
    
    for i in range(len(S)):
        nova_sekvenca = S[:]
        nova_sekvenca[i] = random.choice(list(graf.keys()))  # Nasumično menjanje jednog čvora
        
        # Proveravamo novu sekvencu
        if is_burning_sequence(graf, nova_sekvenca):
            rezultat = evaluate_solution(graf, nova_sekvenca)
            
            # Ako je nova sekvenca bolja, prihvatamo je
            if rezultat < najbolji_rezultat:
                najbolja_sekvenca = nova_sekvenca
                najbolji_rezultat = rezultat

    return najbolja_sekvenca


def perturbation(S):
    """
    Funkcija za perturbaciju sekvence, nasumično menja čvorove.
    """
    nova_sekvenca = S[:]
    i = random.randint(0, len(S) - 1)
    nova_sekvenca[i] = random.choice(list(S))  # Nasumična promena jednog čvora
    return nova_sekvenca

def evaluate_solution(graf, S):
    """
    Evaluacija trenutne sekvence na osnovu broja zapaljenih čvorova.
    """
    return len(S)  # Prosto ocenjujemo po dužini sekvence (možeš dodati i drugi kriterijum ako želiš)


def ILS(graf, max_iter=100):
    """
    ILS algoritam za Graph Burning Problem.
    """
    # Početna sekvenca pomoću pohlepnog algoritma
    S = greedy_burning(graf)
    
    # Provera da li sekvenca pokriva ceo graf
    if not is_burning_sequence(graf, S):
        print("Početna sekvenca nije validna!")
        return None

    najbolja_sekvenca = S
    najbolji_rezultat = evaluate_solution(graf, S)
    
    for _ in range(max_iter):
        # Lokalna pretraga
        S = local_search(graf, S)
        
        # Evaluacija novog rešenja
        rezultat = evaluate_solution(graf, S)
        
        # Ako je novo rešenje bolje, prihvati ga
        if rezultat < najbolji_rezultat and is_burning_sequence(graf, S):
            najbolja_sekvenca = S
            najbolji_rezultat = rezultat
        else:
            # Ako nije bolje, pertubiraj sekvencu
            S = perturbation(S)
        
        # Provera nakon perturbacije da li sekvenca pokriva ceo graf
        if not is_burning_sequence(graf, S):
            print("Sekvenca nakon perturbacije nije validna!")
            break
    
    return najbolja_sekvenca





In [55]:
startTime = time.time()
#filename= "graf.mtx"  
filename = "graf.mtx"

graph = load_graph_from_mtx(filename)
burn_order = greedy_burning(graph)

print("Redoslijed paljenja čvorova:", burn_order)
print("Size",len(burn_order))
print("--- %s sekundi ---" % (time.time() - startTime))



pocetni: 1
nakon prvog koraka u petlji
sekvenca> [1]
zapaljeni: {1, 2, 3, 4, 5}
nezapaljeni {6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}
-----
buduci: [6, 10, 7, 8, 9]
br buduci: 5
-----
kandidati: [15, 16, 17]
naj 15
------
sekvenca> [1, 15]
zapaljeni: {1, 2, 3, 4, 5, 15}
nezapaljeni {6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17}
br zapaljenih: 6
br nezapaljenih: 11
nakon prvog koraka u petlji
sekvenca> [1, 15]
zapaljeni: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 16}
nezapaljeni {12, 13, 14, 17}
-----
buduci: [12, 13, 14]
br buduci: 3
-----
kandidati: [17]
naj 17
------
sekvenca> [1, 15, 17]
zapaljeni: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 16, 17}
nezapaljeni {12, 13, 14}
br zapaljenih: 14
br nezapaljenih: 3
nakon prvog koraka u petlji
sekvenca> [1, 15, 17]
zapaljeni: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}
nezapaljeni set()
-----
buduci: []
br buduci: 0
-----
Redoslijed paljenja čvorova: [1, 15, 17]
Size 3
--- 0.0020029544830322266 sekundi ---


In [56]:
najbolja_sekvenca = ILS(graph)
if najbolja_sekvenca:
    print("Najbolja sekvenca:", najbolja_sekvenca)

pocetni: 1
nakon prvog koraka u petlji
sekvenca> [1]
zapaljeni: {1, 2, 3, 4, 5}
nezapaljeni {6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}
-----
buduci: [6, 10, 7, 8, 9]
br buduci: 5
-----
kandidati: [15, 16, 17]
naj 15
------
sekvenca> [1, 15]
zapaljeni: {1, 2, 3, 4, 5, 15}
nezapaljeni {6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17}
br zapaljenih: 6
br nezapaljenih: 11
nakon prvog koraka u petlji
sekvenca> [1, 15]
zapaljeni: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 16}
nezapaljeni {12, 13, 14, 17}
-----
buduci: [12, 13, 14]
br buduci: 3
-----
kandidati: [17]
naj 17
------
sekvenca> [1, 15, 17]
zapaljeni: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 16, 17}
nezapaljeni {12, 13, 14}
br zapaljenih: 14
br nezapaljenih: 3
nakon prvog koraka u petlji
sekvenca> [1, 15, 17]
zapaljeni: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}
nezapaljeni set()
-----
buduci: []
br buduci: 0
-----


TypeError: unsupported operand type(s) for +: 'enumerate' and 'int'